# Model Training for Real Time Communication through AI for Specially Abled
### Loading the Dataset & Image Data Generation

In [1]:
pwd

'/home/wsuser/work'

In [2]:
!pip install tensorflow==2.7.1

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


2022-11-16 13:30:26.110582: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ibm/dsdriver/lib:/opt/oracle/lib:/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow


In [4]:

# Initializing data augmentation to training variable

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
# Initializing data augmentation to testing variable

test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='eYVV6ov1vL34-l0PJ_Hv1dCPNTybGvofyTi2TUoY8DB1',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'realtimecommunicationsystempowere-donotdelete-pr-lrjheqt6leegjt'
object_key = 'Dataset.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [6]:
# Unzip the Dataset Zip File
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_2.read()), 'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [7]:
pwd

'/home/wsuser/work'

In [8]:
%%bash
ls Dataset

test_set
training_set


In [9]:
# Data augmentation on training data

x_train = train_datagen.flow_from_directory('Dataset/training_set',
                                           target_size=(64,64),
                                           class_mode='categorical',
                                           batch_size=300,
                                            color_mode = "grayscale")
#Data augmentation on testing

x_test = test_datagen.flow_from_directory('Dataset/test_set',
                                           target_size=(64,64),
                                           class_mode='categorical',
                                           batch_size=300,
                                            color_mode = "grayscale")

Found 15750 images belonging to 9 classes.
Found 2250 images belonging to 9 classes.


In [10]:
print("Len x-train : ", len(x_train))
print("Len x-test : ", len(x_test))

Len x-train :  53
Len x-test :  8


In [11]:
# The Class Indices in Training Dataset
x_train.class_indices

{'Come': 0,
 'Good': 1,
 'Hope': 2,
 'Little': 3,
 'One': 4,
 'See': 5,
 'Silent': 6,
 'Washroom': 7,
 'What': 8}

### Model Creation

In [12]:
# Import req. lib

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout

In [13]:
# Building CNN Block

model = Sequential() # Initializing the model
model.add(Convolution2D(32,(3,3),activation='relu',input_shape=(64,64,1))) # Covolution layer
model.add(MaxPooling2D(pool_size=(2,2))) # Max pooling layer
model.add(Flatten()) # Flatten layer
model.add(Dense(units= 512 ,activation='relu')) # Hidden layer 1

model.add(Dense(units = 9,activation='softmax')) # Output layer

2022-11-16 13:30:54.291089: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ibm/dsdriver/lib:/opt/oracle/lib:/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow
2022-11-16 13:30:54.291157: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


In [14]:
# Compiling the Model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [15]:
# Fitting the Model Generator
model.fit_generator(x_train,steps_per_epoch=len(x_train),epochs=10,validation_data=x_test,validation_steps=len(x_test))

/tmp/wsuser/ipykernel_1204/1042518445.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,steps_per_epoch=len(x_train),epochs=10,validation_data=x_test,validation_steps=len(x_test))


Epoch 1/10
53/53 [==============================] - 67s 1s/step - loss: 0.4738 - accuracy: 0.8485 - val_loss: 0.2539 - val_accuracy: 0.9547
Epoch 2/10
53/53 [==============================] - 69s 1s/step - loss: 0.0723 - accuracy: 0.9804 - val_loss: 0.2651 - val_accuracy: 0.9604
Epoch 3/10
53/53 [==============================] - 67s 1s/step - loss: 0.0327 - accuracy: 0.9928 - val_loss: 0.2720 - val_accuracy: 0.9738
Epoch 4/10
53/53 [==============================] - 68s 1s/step - loss: 0.0212 - accuracy: 0.9953 - val_loss: 0.2429 - val_accuracy: 0.9751
Epoch 5/10
53/53 [==============================] - 67s 1s/step - loss: 0.0128 - accuracy: 0.9970 - val_loss: 0.2538 - val_accuracy: 0.9742
Epoch 6/10
53/53 [==============================] - 67s 1s/step - loss: 0.0090 - accuracy: 0.9982 - val_loss: 0.2786 - val_accuracy: 0.9747
Epoch 7/10
53/53 [==============================] - 68s 1s/step - loss: 0.0058 - accuracy: 0.9990 - val_loss: 0.2997 - val_accuracy: 0.9760
Epoch 8/10
53/53 [==

### Saving the Model

In [16]:
model.save('IBM_Communication_Model.h5')
# Current accuracy is 0.9756

In [17]:
# Convert the Saved Model to a Tar Compressed Format
!tar -zcvf IBM_TrainedModel.tgz IBM_Communication_Model.h5

IBM_Communication_Model.h5


In [18]:
%%bash
ls -ll

total 354496
drwxrwx--- 4 wsuser wscommon      4096 Nov 16 13:24 Dataset
-rw-rw---- 1 wsuser wscommon 189041736 Nov 16 13:43 IBM_Communication_Model.h5
-rw-rw---- 1 wsuser wscommon 173953130 Nov 16 13:43 IBM_TrainedModel.tgz


### Watson Machine Learning

In [19]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 20.2 MB/s eta 0:00:01


In [21]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "1pU54wwcdvBmrkPO8xfzURxezUm1kSSS4LqcGW9UYnAX"
}

client = APIClient(wml_credentials)

### Save to Deployment Space

In [22]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return (next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [23]:
space_uid = guid_from_space_name(client, 'Real-Time Communication System Powered by AI for Specially Abled')
print("Space UID : ", space_uid)

Space UID :  6a01112a-bdfb-49e7-85b9-74467d58d0dd


In [24]:
client.set.default_space(space_uid)

'SUCCESS'

In [25]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [27]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [32]:
model_details = client.repository.store_model(model='IBM_TrainedModel.tgz', meta_props={
    client.repository.ModelMetaNames.NAME: "CNN",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.7"})
model_id = client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


In [33]:
model_id

'e987aab4-35be-4859-a79f-2e88420cce35'

In [34]:
client.repository.download(model_id,"IBM_Model_Download.tar.gz")

Successfully saved model content to file: 'IBM_Model_Download.tar.gz'


'/home/wsuser/work/IBM_Model_Download.tar.gz'